# Sample Text2SQL Agent Cleanup
In this section we clean up the resources created for the sample Text2SQL agent

#### Ensure the latest version of boto3 is shown below

In [16]:
!pip freeze | grep boto3

boto3==1.37.2


#### Load in environment variables to notebook

In [18]:
# Retrieve import path
%store -r IMPORTS_PATH

# Retrieve account info
%store -r region

# Retrive relevant resources
%store -r base_bucket_name
%store -r athena_results_bucket_name
%store -r athena_database_name
%store -r text2sql_lambda_function_name

#### Retrieve imports environment variable and bring libraries into notebook

In [19]:
%run $IMPORTS_PATH

Successfully imported necessary libraries into notebook


#### Define Clients

In [20]:
agents = AgentsForAmazonBedrock()
s3_client = boto3.client('s3', region)
lambda_client = boto3.client('lambda', region)

#### Destroy Sample Text2SQL Agent

In [21]:
agents.delete_agent("sample-text2sql-agent",delete_role_flag=True)

Deleting alias 2DY21TOOXE from agent SNKOEJYUCZ
Deleting alias TSTALIASID from agent SNKOEJYUCZ


#### Destroy Athena database

In [24]:
def execute_query(query, database):
    athena_client = boto3.client('athena')
    return athena_client.start_query_execution(
        QueryString=query,
        QueryExecutionContext={'Database': database},
        ResultConfiguration={'OutputLocation': f's3://{athena_results_bucket_name}/'}
    )

def delete_all_tables_and_database():
    athena_client = boto3.client('athena')

    # Get all tables
    tables = [t['Name'] for t in athena_client.list_table_metadata(
        CatalogName='AwsDataCatalog',  # Add this parameter
        DatabaseName=athena_database_name
    )['TableMetadataList']]

    # Drop all tables
    for table in tables:
        execute_query(f"DROP TABLE IF EXISTS `{table}`", athena_database_name)
        print(f"Dropped table: {table}")

    # Drop the database
    execute_query(f"DROP DATABASE IF EXISTS `{athena_database_name}`", 'default')
    print(f"Dropped database: {athena_database_name}")

delete_all_tables_and_database()

Dropped table: frpm
Dropped table: satscores
Dropped table: schools
Dropped database: california_schools


#### Destroy S3 buckets

In [36]:
s3_resource = boto3.resource('s3')

# Delete contents of base bucket
bucket1 = s3_resource.Bucket(base_bucket_name)
bucket1.objects.all().delete()
bucket1.object_versions.all().delete()

# Delete contents of Athena results bucket
bucket2 = s3_resource.Bucket(athena_results_bucket_name)
bucket2.objects.all().delete()
bucket2.object_versions.all().delete()

# Delete the empty buckets
s3_client = boto3.client('s3')
s3_client.delete_bucket(Bucket=base_bucket_name)
s3_client.delete_bucket(Bucket=athena_results_bucket_name)

print(f"Bucket {base_bucket_name} has been successfully deleted")
print(f"Bucket {athena_results_bucket_name} has been successfully deleted")

Bucket text2sql-agent-861276117215-02e05f has been successfully deleted
Bucket text2sql-athena-results-861276117215-7b5bbc has been successfully deleted


#### Destroy Lambda function

In [38]:
lambda_client.delete_function(FunctionName=text2sql_lambda_function_name)
print(f"Successfully deleted Lambda function: {text2sql_lambda_function_name}")

Successfully deleted Lambda function: text2sql
